In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [2]:
def PowerEigDecomp(A,n):

    x = np.ones(A.shape[0]).reshape((A.shape[0],1))

    eig_val = []
    eig_vect = []
    for i in range(n):

        x1 = np.zeros(A.shape[0]).reshape((A.shape[0],1))
        it = 0
        
        while((np.linalg.norm(x1 - x) > 10 ** -5) & (it<300)):
            x1 = np.copy(x)
            x = A@x / np.linalg.norm(A@x)
            it += 1

        l = ((A@x).T@x / x.T@x).reshape(1)[0]
        ev = np.copy(x).reshape(A.shape[0])
       
        print(str(i)+" th dominant eigmap found")

        eig_val.append(l)
        eig_vect.append(ev)

        A = A - eig_val[i]*(x @ x.T)
    
    print(eig_val)
    return eig_val,eig_vect

In [30]:
def SVD(A):
    U = np.zeros((A.shape[0],A.shape[0]))
    S = np.zeros((A.shape[0],A.shape[1]))
    V = np.zeros((A.shape[1],A.shape[1]))
    
    eigval_A,eigvect_A = PowerEigDecomp(A@A.T,A.shape[0])

    eigval_AT,eigvect_AT = PowerEigDecomp(A.T@A,A.shape[1])

    for i in range(len(eigval_A)):
        S[i,i] = eigval_A[i]**0.5
    
    
    for i in range(len(eigval_A)):
        U[:,i] = eigvect_A[i]
    
    for i in range(len(eigval_AT)):
        V[i,:] = eigvect_AT[i]
    
    
    return U,S,V

In [11]:
def reconstruct(A,k):
    
    U_A,S_A,V_A = SVD(A)
    A_hat = U_A[:,:k] @ S_A[:k,:k] @ V_A.T[:k,]

    return A_hat

In [12]:
def FrobeniusNorm(A):
    
    ans = 0
    
    for x in A:
        for y in x:
            ans += y**2
    
    return ans ** 0.5

In [6]:
df = pd.read_csv("train.csv")

df

# img = np.array(df.head())
# img

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,0,618,2,3,1,0,0,0,0,0,...,0,2,0,2,1,1,1,54,0,1
1,1,608,0,0,0,0,0,1,3,0,...,1,0,0,1,0,0,41,23,0,2
2,2,664,0,1,0,0,1,0,0,0,...,0,0,1,0,2,0,1,2,18,2
3,3,688,0,0,0,0,0,0,0,2,...,0,1,0,0,0,21,0,18,0,2
4,4,642,0,0,0,1,1,0,0,1,...,0,1,0,0,0,1,32,17,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59995,614,2,0,0,1,1,0,2,2,...,1,1,3,1,0,1,2,36,1,3
59996,59996,633,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,35,23,0,1
59997,59997,648,0,1,1,0,1,2,0,0,...,0,0,1,0,0,1,1,35,3,3
59998,59998,639,0,0,0,0,1,1,1,2,...,2,1,0,0,0,0,12,5,2,1


In [ ]:
X = df.values[:,2:]
k = 256
recon_err = []
for i in range(k):
    X_hat = reconstruct(X,i)
    recon_err.append(FrobeniusNorm(X_hat-X))

In [33]:
X = np.random.randint(0,9,(5,3))
k = 3
recon_err = []
for i in range(k):
    X_hat = reconstruct(X@X.T,i)
    recon_err.append(FrobeniusNorm(X_hat-X))

0 th dominant eigmap found
1 th dominant eigmap found
2 th dominant eigmap found
3 th dominant eigmap found
4 th dominant eigmap found
[1357869.0067103314, -5431476.026841326, 21725904.107365303, -86903616.42946118, 347614465.7178447]
0 th dominant eigmap found
1 th dominant eigmap found
2 th dominant eigmap found
3 th dominant eigmap found
4 th dominant eigmap found
[1357869.0067103314, -5431476.026841326, 21725904.107365303, -86903616.42946118, 347614465.7178447]
[[0.37350412 0.37350412 0.37350412 0.37350412 0.37350412]
 [0.408755   0.408755   0.408755   0.408755   0.408755  ]
 [0.35538965 0.35538965 0.35538965 0.35538965 0.35538965]
 [0.52915732 0.52915732 0.52915732 0.52915732 0.52915732]
 [0.53582157 0.53582157 0.53582157 0.53582157 0.53582157]]
[[0.37350412 0.408755   0.35538965 0.52915732 0.53582157]
 [0.37350412 0.408755   0.35538965 0.52915732 0.53582157]
 [0.37350412 0.408755   0.35538965 0.52915732 0.53582157]
 [0.37350412 0.408755   0.35538965 0.52915732 0.53582157]
 [0.373

C:\Users\Joel Suvisesha Muthu\AppData\Local\Temp\ipykernel_9136\1780324484.py:11: RuntimeWarning: invalid value encountered in scalar power
  S[i,i] = eigval_A[i]**0.5


ValueError: operands could not be broadcast together with shapes (5,5) (5,3) 

In [ ]:
plt.plot([x for x in range(k)],recon_err
         ,marker = 'o'
         , markerfacecolor = 'yellow'
         , color = 'black')
plt.xlabel("k")
plt.ylabel("reconstruction error")
plt.title("reconstruction error vs k")